## INTRODUCTION
  In Orellana *et al.* 2016, we have used all the crystal structures of a protein to create an ensemble. What we call an ensemble in this article is a set of structures that are not chimeric, represent the whole protein rather than a single domain of it, and do not contain more than 5 consecutive non-terminal missing residues. As much as we would like to automate this process, preparation of an ensemble often requires manual intervention due to problems like, chimeric constructs or the placement of subunits (clockwise vs. anticlockwise). There is also a decision to be made whether to exclude a structure that has non-terminal missing residues.
  
  While preparing the ensembles for this article, we have taken care of those problems manually and made a decision to repair the structures  if it was known to represent an important state which does not have any other representatives. Altough it very much depends on the state structure, and the location of the missing residues, we recommend repairing structures with only less than 5 consecutive residues missing. There is an example modeller script that can be used for this.  It is upto the user to fix and include them in the ensemble later.
  
  This tutorial will be using GLIC as an example. Because this was the most challenging ensemble we prepared, it highlights most of the issues the users are likely to encounter. 

## HOW TO
### Requirements
Before we get started make sure you have installed the enspdb package, or you are running this notebook in a place where enspdb is the subfolder. You will find the other required python packages within [github](https://github.com/ozyo/enspdb) README.md

To determine the missing residues, enspdb relies on sequence alignment. If you use enspdb to prepare a start and an end structure for eBDIMs then the alignment is generated via BioPython. However if you are preparing an ensemble you must install Clustal Omega or run the provided fasta sequence through a multiple sequence alignment tool and save the alignment in fasta format. The installation of Clustal Omega can easily be done via conda. After you install it you should provide the full path for the clustal executable.

Altough not a requirement, if you want to fix any broken structures you will need a modelling program. We have provided an example script for MODELLER.

### Obtaining PDB Files
  It can be daunting to find all the available PDB files, and the chains that actually belongs to your protein. The easiset way to do this is to go through the UniProtKB database and enspdb requires the UniProt ID to prepare the ensemble. 
  ![title](img/uniprot_ini_search.png)
  
After finding the UniProt ID you can run the commands below to gather information. Besides the UniProt ID, you also have to provide the total number of chains that are in a biological assembly. Heteromeric structures require a different routine which are not yet supported in enspdb. However similar logic applies in preperation of such an ensemble that only the shared CA coordinates should be kept, any broken structures should be fixed or excluded.

You should also provide a folder that exits for inputs/outputs to be read/written. If you use "." This will take the current directory for outputs.

In [1]:
from pathlib import Path
query='Q7NDN8' #UniPortKB ID
mer=5 #Total number of chains
cwd=Path("glic_test") #This directory must exist
cwd.mkdir(exist_ok=True)
exclude=None #[List of PDB IDs] 
#If you already know some structures you want to exclude provide them as a list. 

In [2]:
from enspdb import ensemble as pE
from importlib import reload

reload(pE)
info=pE.PDBInfo(query,mer,exclude=exclude,workdir=cwd) #Add the exclude argument to the end.

CRITICAL:root:Cannot process PDB id 3IGQ. It does not contain a complete set


PDBInfo function goes into the UniProt page of the given ID, and retrives the 3D structures table (as seen in figure below) and the reference sequence to be used for the alignment. For the reasons below it is much easier to prepare the ensemble via a database like UniProtKB:
* Because there is a possiblity of all the structures having the same missing residues, the reference sequence is a good control to have. 
* Within 3D structure table, only the ones with the label "X-ray" are downloaded. Currently we don't support multi-model pdb files.
* The relevant chains for this protein is also extracted from this table, this is much easier than reading PDB headers.

In the example above, you already see that 3IGQ structure is skipped. That is because this file had 6 subunits vs. 5. If you go into the RCSB Database and look at this structure you will see that it contains only the extracellular domain and not the whole structure. To our advantage this one had more subunits than we required, so it is eliminated early on.

![title](img/uniprot_3dtable.png)

  After the information is gathered, info object is populated with the possible structures to be used and the reference sequence. You can reach the list via commands below. If you want to manipulate this list, add or remove structures, or change the reference sequence you can do so at this stage. Please not that all the functions in prepENS module uses and modifes this class directly.

In [3]:
info.query #UniPort ID used to initialize this class
#and it is also used for the output names.
print(info.mer) # Total number of chains
print(info.result) #Contains the information from the 3D structures table.
print(info.refseqs) #Contains the reference sequence. 

5
{'2XQ3': [1, [['A', 'B', 'C', 'D', 'E']]], '2XQ4': [1, [['A', 'B', 'C', 'D', 'E']]], '2XQ5': [1, [['A', 'B', 'C', 'D', 'E']]], '2XQ6': [1, [['A', 'B', 'C', 'D', 'E']]], '2XQ7': [1, [['A', 'B', 'C', 'D', 'E']]], '2XQ8': [1, [['A', 'B', 'C', 'D', 'E']]], '2XQ9': [1, [['A', 'B', 'C', 'D', 'E']]], '2XQA': [1, [['A', 'B', 'C', 'D', 'E']]], '3EAM': [1, [['A', 'B', 'C', 'D', 'E']]], '3EHZ': [1, [['A', 'B', 'C', 'D', 'E']]], '3EI0': [1, [['A', 'B', 'C', 'D', 'E']]], '3LSV': [1, [['A', 'B', 'C', 'D', 'E']]], '3P4W': [1, [['A', 'B', 'C', 'D', 'E']]], '3P50': [1, [['A', 'B', 'C', 'D', 'E']]], '3TLS': [1, [['A', 'B', 'C', 'D', 'E']]], '3TLT': [1, [['A', 'B', 'C', 'D', 'E']]], '3TLU': [1, [['A', 'B', 'C', 'D', 'E']]], '3TLV': [1, [['A', 'B', 'C', 'D', 'E']]], '3TLW': [1, [['A', 'B', 'C', 'D', 'E']]], '3UU3': [1, [['A', 'B', 'C', 'D', 'E']]], '3UU4': [1, [['A', 'B', 'C', 'D', 'E']]], '3UU5': [1, [['A', 'B', 'C', 'D', 'E']]], '3UU6': [1, [['A', 'B', 'C', 'D', 'E']]], '3UU8': [1, [['A', 'B', 'C', 'D

  The next command downloads the structures from the RCSB database. Depending on the number of structures this function might take a while to finish because after it downloads the files it goes through the steps below to clean them:
  1. It checks for lines with TITLE to find the chimeric structures.
  2. It seperates the biological assemblies within the same PDB file
  3. It keeps only the CA coordinates, which are the only relevant atoms for comparison with eBDIMs trajectories. Obtaining only CA coordinates is also a way to clean the PDB files from bound ligands, ions, waters, etc.
  4. It extracts the sequence and the residue numbering from the CA coordinates and writes them to text files.

enspdb relies on multiple sequence alignment to determine the missing residues. This also allows us to ignore the differences in residue numbering between different structures, non-consecutive numbering. The residue numbering information is only used to extract the residues from the the PDB file itself. 
The sequence for the alignment is extracted from CA coordinates, rather than PDB header. Because a residue might have missing CA coordinate, despite being listed in the sequence records.

  In the example below you see that there are more warnings, stating that more structures will be excluded from the ensemble because they are chimeras. The function checks for two keywords in the title 'CHIMERA' and 'FUSED'. Since these structures cannot be used, their information is removed from the info.result dictionary automatically and those files are deleted.

In [19]:
reload(pE)
pE.PDBInfo.downloadPDB(info,cwd)
pE.PDBInfo.process_pdbs(info,cwd, overwrite_pdb=True)

### Multiple Sequence Alignment and Identification of the Shared Region
If you check the output folder, you will find two text files: "_seq.txt" and "_residmap.txt". We need the information within these files for the multiple sequence alignment and the subsequent cleaning of the terminal residues that are not shared between the structures. It is possible that some structures still contain a terminal tag, that would interfere with the alignment and introduce unncessary gaps. Any chain that contains a non-terminal gap at this stage is marked as broken and the assembly they belong to is skipped.

  If you chose to run the alignment somwhere else, you must clean any headers that is written to it. And the names in the fasta header must be in the same format as in the example file in this tutorial. The alignment should also contain a reference sequence with a header refseq. When you run the function below with the argument alnf="alignmentfile.fasta" , it will only go through the file to determine the broken chains. It does not run clustal, so you don't have to provide it. The alignment file must be placed where *cwd* is set to.

In [20]:
clustalopath="/home/ozyo/.local/bin/clustalo-1.2.4-Ubuntu-x86_64"
alignments=pE.PDBInfo.msa_clustal(info,cwd,clustalopath)

Found blocks: {'B1': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217

In [21]:
#info.core_show(cwd)
#reload(pE)
pE.PDBInfo.get_core(info,cwd)

In [ ]:
# Kernel often crashes when executing modeller through this cell.
%cd glic_test/clean_pdbs
%cp ../../fix_model.py ../../myloop.py ../../seg.ali .
%run -i 'fix_model.py' #You have to modify the contents of this file if you are working on your own example.
%cp 4NPQ_4_fixed.pdb 4NPQ_4.pdb 

In [22]:
info.broken.remove('4NPQ_4.pdb')
#Python indexing starts from 0
chains=info.result['4NPQ'][1][3] 
#Chain informations are the 2nd element and within that list the 4th ensemble. 
info.coremer['4NPQ_4.pdb']=chains
info.coreresids.loc['4NPQ_4.pdb|S|']=info.coreresids.loc['4NPQ_4.pdb|T|'] 
#Since this is a homopentamer, I have just used the information from other chain to set the residue numbers for |S|

The function is a class method, thus we don't need to any arguments to it, only the path to the clustal omega executable. So if you need to change names of the sequence file, for example, you can do so by modifying the class object directly, info.seqfilename is mutable. If you need to add or delete chains/structures you must modify both the text files and the info.result dictionary. You will see an example below

As you see above there are quite a few chains that have missing residues or insertions. All these identified broken chains are stored in info.broken object. This list is used in the subsequent function that extracts the shared CA region. But before we do that we should take a look at the alignment file. 

Altough the assemblies with broken chains will be skipped, the CA coordinates and the FASTA alignment information of those are kept. For example from the fasta alignment file, we saw that chain S in 4NPQ PDB file, assembly 4 has two missing residues. But the rest of the structures in this assembly is intact. Because there are not many "possible closed state" structures of GLIC, we decided to repair this assembly. You can use the example MODELLER script to rebuild those missing residues. After you have fixed the structure you should follow the steps below:
1. Modify the info.broken object to exclude that chain. 
2. You must also add it back to the info.coreresids and info.coremer.
3. You must rename the final, fixed file to it's original 4NPQ_4.pdb

Now that we have all the structures we want to use ready, and the list of broken structures is updated we can move on to the next step to extract the common residues. As you can see for GLIC some files does not have the initial valine and serine residues ın the N termini. Some of the structures also have a missing residue in the C termini. The function below, uses the information returned from the msa function above. It finds which residues should be removed from the N and C termini by mapping back the sequence to the residue numbers written to "_residmap.txt". 
The next and the last step is to align the structures.

In [24]:
from MDAnalysis import Universe
from MDAnalysis.analysis import align
import pandas as pd

def align_get_rmsd(ref_id:str,cwd:Path):
    ref = Universe(f"{cwd}/clean_pdbs/correct_{ref_id}.pdb")
    data = {"pdb_name":[],"rmsd":[],"universes":[]}
    for pdb in info.coremer:
        if pdb in info.broken:
            continue
        u=Universe(f"{cwd}/clean_pdbs/correct_{pdb}")
        print(pdb)
        a = align.AlignTraj(u,ref,in_memory=True,strict=True)
        a.run()
        data["pdb_name"].append(pdb)
        data["universes"].append(a)
        data["rmsd"].append(a.results.rmsd[0])
    data=pd.DataFrame(data)
    print(data[data.rmsd > 5])
    return data
rmsd = align_get_rmsd("4NPQ_1",cwd)

3TLW_1.pdb
2XQ8_1.pdb
4LMK_1.pdb
5HCM_1.pdb
2XQA_1.pdb
6HJZ_2.pdb
5V6O_1.pdb
6F0N_1.pdb
6HZW_1.pdb
3TLS_1.pdb
4HFI_1.pdb
4NPP_1.pdb
5MZR_1.pdb
3EI0_1.pdb
6F12_1.pdb
6HY9_1.pdb
3TLU_1.pdb
6F0U_1.pdb
6HY5_1.pdb
4IL4_1.pdb
6F16_1.pdb
6HJB_1.pdb
3UUB_2.pdb
6HYV_1.pdb
4F8H_1.pdb
6F0M_1.pdb
6F13_1.pdb
3P50_1.pdb
6HJA_1.pdb
5NKJ_1.pdb
3UU8_1.pdb
2XQ6_1.pdb
5HEG_1.pdb
6HJ3_1.pdb
4HFB_1.pdb
6F15_1.pdb
5J0Z_1.pdb
6F11_1.pdb
6HYX_1.pdb
3TLV_1.pdb
4ZZC_1.pdb
3UU6_1.pdb
6F0V_1.pdb
6F0R_1.pdb
3UUB_1.pdb
3EAM_1.pdb
5MZQ_1.pdb
4ILC_1.pdb
5MZT_1.pdb
6HYA_1.pdb
6I08_1.pdb
6HYR_1.pdb
6HPP_1.pdb
5MVN_1.pdb
6F0Z_1.pdb
4HFC_1.pdb
5L47_1.pdb
5HEH_1.pdb
2XQ5_1.pdb
3UU5_1.pdb
4HFE_1.pdb
6HZ1_1.pdb
6F0J_1.pdb
4LMJ_1.pdb
3EHZ_1.pdb
4NPQ_4.pdb
4LML_1.pdb
5NJY_1.pdb
5MUO_1.pdb
5V6N_1.pdb
6HJI_1.pdb
4ZZB_1.pdb
4ILB_1.pdb
4QH4_1.pdb
3UU4_1.pdb
4IRE_1.pdb
6F7A_1.pdb
5HCJ_1.pdb
5MVM_1.pdb
3TLT_1.pdb
4HFD_1.pdb
4QH5_1.pdb
4NPQ_1.pdb
6F0I_1.pdb
6HYW_1.pdb
5L4H_1.pdb
6HJZ_1.pdb
2XQ4_1.pdb
4IL9_1.pdb
5IUX_1.pdb
6HYZ_1.pdb

As you can see in the above analysis we have quite a few PDB files that show a high RMSD difference to the reference. They are outliers and most of the time you have to examine what's wrong manually in each file and either discard or repair them. GLIC has 5 subunits arranged in counter-clockwise order, usually, however, the PDB files above do not contain chains in the counter-clockwise order and their order needs to be changed.

In [38]:
from enspdb import chorder as ch
from importlib import reload
reload (ch)

print(ch.reorder({k:v for k,v in info.coremer.items() if k not in info.broken},cwd/"clean_pdbs",tag="correct_"))

rmsd = align_get_rmsd("4NPQ_1",cwd)

IndexError: tuple index out of range

In [34]:
from enspdb import chorder as ch
reload(ch)
forced={'4LMK_1.pdb':["A","E","D","C","B"],'4LML_1.pdb':["A","C","B","E","D"],'5V6N_1.pdb':["A","E","D","C","B"],'4LMJ_1.pdb':['A','E','D','C','B']}
ch.forceorder({k:v for k,v in info.coremer.items() if k not in info.broken},cwd/"clean_pdbs",ordlist=['5V6N_1.pdb','4LML_1.pdb','4LMK_1.pdb','4LMJ_1.pdb'],tag="correct_",forordict=forced, alph=False)
ch.forceorder({k:v for k,v in info.coremer.items() if k not in info.broken},cwd/"clean_pdbs",ordlist=['5V6O_1.pdb','6I08_1.pdb','4LMJ_1.pdb'],tag="correct_", alph=True)
ref = Universe("glic_test/clean_pdbs/correct_4NPQ_1.pdb")
data = {"pdb_name":[],"rmsd":[],"universes":[]}
for pdb in info.coremer:
    if pdb in info.broken:
        continue
    u=Universe(f"glic_test/clean_pdbs/correct_{pdb}")
    a = align.AlignTraj(u,ref,in_memory=True)
    a.run()
    u.select_atoms("all").write(f"glic_test/clean_pdbs/correct_{pdb}")
    data["pdb_name"].append(pdb)
    data["universes"].append(u)
    data["rmsd"].append(a.results.rmsd[0])
data=pd.DataFrame(data)
data[data.rmsd > 5]

/home/ozyo/.local/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:429: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"
/home/ozyo/.local/lib/python3.9/site-packages/MDAnalysis/coordinates/base.py:892: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")
/home/ozyo/.local/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "


,pdb_name,rmsd,universes
2,4LMK_1.pdb,25.757104,<Universe with 1540 atoms>
6,5V6O_1.pdb,25.695513,<Universe with 1540 atoms>
50,6I08_1.pdb,18.429112,<Universe with 1540 atoms>
63,4LMJ_1.pdb,25.832275,<Universe with 1540 atoms>
66,4LML_1.pdb,25.742117,<Universe with 1540 atoms>
69,5V6N_1.pdb,25.789068,<Universe with 1540 atoms>


### Structure Alignment
Since we have extracted a common region, and CA coordinates only, all the structures have the same number of atoms. Regardless of the point mutations and the absolute residue number differences we can use almost any alignment tool to superimpose them. For this step you can use your favorite visualizer. In fact it is a good idea to look at the ensemble, and check each structure individually. For example some GLIC structures have a different subunit placement: anticlockwise vs. clockwise. This makes it impossible to superimpose. To fix this issue you could reorder and rename the chains so that the placement matches the rest of the ensemble.

When you are finished the alignment of the structures, all you need to do is to combine them, then your ensemble is ready for uploading to the server.Since eBDIMs requires MODEL and END lines in the beginning and the end of the file, you can use a bash loop similar to this to combine them. Since all the structures have the same number of atoms, you can do the alignment with this combined file rather then individual files.

In [37]:
%%bash
touch glic_test/GLIC_ensemble.pdb
for pdb in `ls -1 glic_test/clean_pdbs/correct*pdb`
do
echo "MODEL" >> glic_test/GLIC_ensemble.pdb
cat $pdb >> glic_test/GLIC_ensemble.pdb
echo "END" >> glic_test/GLIC_ensemble.pdb
done

<table><tr>
<td> <img src="img/side.png" alt="Drawing" style="width: 100%;"/> </td>
<td> <img src="img/top.png" alt="Drawing" style="width: 100%;"/> </td>
</tr></table>

If you have any questions do not hesitate to contact us. enspdb, ensemble preperation is a package being developed. Watch out for new features on [github](https://github.com/ozyo/enspdb)